In [4]:
from eurostatapiclient import EurostatAPIClient
import pandas as pd
import sys
import os

In [5]:
project_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_path not in sys.path:
    sys.path.append(project_path)

import utils.get_data_utils as gdu

In [6]:
# Get the list of years for analysis
time_period = [ str(t) for t in range(2005, 2024)]

# Get the list of countries for analysis
country_codes = [
    "BE",  # Belgium
    "EL",  # Greece
    "LT",  # Lithuania
    "PT",  # Portugal
    "BG",  # Bulgaria
    "ES",  # Spain
    "LU",  # Luxembourg
    "RO",  # Romania
    "CZ",  # Czechia
    "FR",  # France
    "HU",  # Hungary
    "SI",  # Slovenia
    "DK",  # Denmark
    "HR",  # Croatia
    "MT",  # Malta
    "SK",  # Slovakia
    "DE",  # Germany
    "IT",  # Italy
    "NL",  # Netherlands
    "FI",  # Finland
    "EE",  # Estonia
    "CY",  # Cyprus
    "AT",  # Austria
    "SE",  # Sweden
    "IE",  # Ireland
    "LV",  # Latvia
    "PL"   # Poland
]


# Get the list of indicators along with their filters for our analysis
indicators = {
    'prc_hpi_a': {
        "unit": "RCH_A_AVG", # Annual average rate of change
        "purchase": "TOTAL"
    },  # House price index
    'tipsho60': {
        "unit": "PTIR_LT_AVG" # Price-to-income ratio relative to long-term average
    }, # Standardised house price-to-income ratio
    'sts_cobp_a': {
        "unit": "I21", # Index, 2021=100
        "indic_bt": "BPRM_SQM", # Building permits - m2 of useful floor area
        "cpa2_1": "CPA_F41001" # Residential buildings
    }, # Building permits
    'migr_imm1ctz': {
        "citizen": "TOTAL",
        "agedef": "REACH",
        "age": "TOTAL",
        "sex": "T"
    }, # Immigration by age group, sex and citizenship
    'ilc_lvps08': {
        "sex": "T",
        "age": "Y18-34" # From 18 to 34 years
    }, # Persons living with their parents or contributing/benefiting from the household income (population aged 18 to 34 years)
    'sts_copi_a': {
        "unit": "PCH_SM", # Percentage change compared to same period in previous year
        "indic_bt": "PRC_PRR", # Producer prices
    }, # Construction producer prices or costs, new residential buildings
    # 'crim_off_cat': {
    #     "unit": "P_HTHAB" # Per hundred thousand inhabitants
    # }, # Police-recorded offences by offence category
    'earn_nt_net': {
        "ecase": "P1_NCH_AW100", # Single person without children earning 100% of the average earning
        "currency": "EUR",
        "estruct": "NET" # Net earning
    }, # Annual net earnings
    'ilc_li48': {
        "deg_urb": "TOTAL"
    }, # At-risk-of-poverty rate after deducting housing costs by degree of urbanisation
    'tps00003': {
        # No aditional parameters
    }, # Population density
    'tps00206': {
        # No aditional parameters
    }, # Crude marriage rate
    'tec00118': {
        # No aditional parameters
    }, # HICP - inflation rate
    'tps00019': {
        "indic_de": "GROWRT" # Crude rate of total population change
    }, # Population change - crude rates of total change, natural change and net migration plus adjustment
    'tesem120': {
        "sex": "T"
    }, # Unemployment rate by sex
    # 'ert_bil_eur_a': {
    #     "unit": ""
    # }, # Euro/ECU exchange rates - annual data
}

renamingIndicatorsMapping = {
    'prc_hpi_a': "house_price_index",      # House price index
    'tipsho60': "price_income_ratio",       # Price-to-income ratio
    'sts_cobp_a': "building_permits",        # Building permits (sqm useful floor area)
    'migr_imm1ctz': "immigration",           # Immigration demographics
    'ilc_lvps08': "living_with_parents",     # Living with parents (18-34)
    'sts_copi_a': "construction_prices",     # Construction producer prices/costs
    'earn_nt_net': "net_earnings",           # Net earnings
    'ilc_li48': "poverty_rate",              # At-risk-of-poverty rate
    'tps00003': "pop_density",               # Population density
    'tps00206': "marriage_rate",             # Crude marriage rate
    'tec00118': "inflation_rate",            # HICP - inflation rate
    'tps00019': "pop_change",                # Population change rate
    'tesem120': "unemployment_rate"          # Unemployment rate
}


for indicator in indicators:
    indicators[indicator]['geo'] = country_codes
    indicators[indicator]['time'] = time_period
    indicators[indicator]['freq'] = 'A'

print(indicators)

{'prc_hpi_a': {'unit': 'RCH_A_AVG', 'purchase': 'TOTAL', 'geo': ['BE', 'EL', 'LT', 'PT', 'BG', 'ES', 'LU', 'RO', 'CZ', 'FR', 'HU', 'SI', 'DK', 'HR', 'MT', 'SK', 'DE', 'IT', 'NL', 'FI', 'EE', 'CY', 'AT', 'SE', 'IE', 'LV', 'PL'], 'time': ['2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023'], 'freq': 'A'}, 'tipsho60': {'unit': 'PTIR_LT_AVG', 'geo': ['BE', 'EL', 'LT', 'PT', 'BG', 'ES', 'LU', 'RO', 'CZ', 'FR', 'HU', 'SI', 'DK', 'HR', 'MT', 'SK', 'DE', 'IT', 'NL', 'FI', 'EE', 'CY', 'AT', 'SE', 'IE', 'LV', 'PL'], 'time': ['2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023'], 'freq': 'A'}, 'sts_cobp_a': {'unit': 'I21', 'indic_bt': 'BPRM_SQM', 'cpa2_1': 'CPA_F41001', 'geo': ['BE', 'EL', 'LT', 'PT', 'BG', 'ES', 'LU', 'RO', 'CZ', 'FR', 'HU', 'SI', 'DK', 'HR', 'MT', 'SK', 'DE', 'IT', 'NL', 'FI', 'EE', '

In [7]:
df = gdu.get_eurostat_data(indicators, renamingIndicatorsMapping)

print(df.head(100))

NameError: name 'get_eurostat_data' is not defined